# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

You should also have deployed a campaign for each of these solution versions. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover the campaigns in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the Movielens dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return a movie by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [7]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [8]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [9]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
True Lies (1994)
Batman (1989)
Clear and Present Danger (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for similar movies render in a nice dataframe. Again, let's create a helper function to achieve this.

In [10]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [11]:
samples = items_df.sample(5)
samples

,title
movieId,
32387,"Sword of Doom, The (Dai-bosatsu tÃ´ge) (1966)"
26084,"Music Man, The (1962)"
5899,Zulu (1964)
5346,Wild Orchid (1990)
44709,Akeelah and the Bee (2006)


In [12]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,"Sword of Doom, The (Dai-bosatsu tÃ´ge) (1966)","Music Man, The (1962)",Zulu (1964),Wild Orchid (1990),Akeelah and the Bee (2006)
0,I Stand Alone (Seul contre tous) (1998),"Pom Poko (a.k.a. Raccoon War, The) (Heisei tan...",Michael Jackson's Thriller (1983),"Shawshank Redemption, The (1994)","World's Fastest Indian, The (2005)"
1,"Yes Men, The (2003)","Miracle of Morgan's Creek, The (1944)",Turbo Kid (2015),Forrest Gump (1994),Under the Tuscan Sun (2003)
2,Wild at Heart (1990),Unfaithfully Yours (1948),The Gift (2015),Pulp Fiction (1994),"Good Night, and Good Luck. (2005)"
3,Visitor Q (Bizita Q) (2001),"Palm Beach Story, The (1942)",Final Flight of the Osiris (2003),"Silence of the Lambs, The (1991)",NaN
4,Forgotten Silver (1996),"Last Waltz, The (1978)","Better Tomorrow, A (Ying hung boon sik) (1986)",Braveheart (1995),NaN
5,DiG! (2004),"Magnificent Ambersons, The (1942)",Filth (2013),"Matrix, The (1999)",NaN
6,Cinemania (2002),Kind Hearts and Coronets (1949),Booty Call (1997),Schindler's List (1993),NaN
7,Palindromes (2004),"Decline of Western Civilization, The (1981)",Barbershop 2: Back in Business (2004),Star Wars: Episode IV - A New Hope (1977),NaN
8,Ken Park (2002),Hands on a Hard Body (1996),Car Wash (1976),Jurassic Park (1993),NaN
9,Sonatine (Sonachine) (1993),"Butcher's Wife, The (1991)",Splendor in the Grass (1961),Terminator 2: Judgment Day (1991),NaN


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [13]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[546, 240, 263]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [14]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [15]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,546,240,263
0,Creepshow (1982),Matilda (1996),Catch Me If You Can (2002)
1,"Texas Chainsaw Massacre, The (1974)",Jingle All the Way (1996),Ocean's Eleven (2001)
2,Hellraiser (1987),Harriet the Spy (1996),"Beautiful Mind, A (2001)"
3,Friday the 13th Part VI: Jason Lives (1986),101 Dalmatians (1996),Bowling for Columbine (2002)
4,Near Dark (1987),Kazaam (1996),Pride & Prejudice (2005)
5,Friday the 13th Part 3: 3D (1982),James and the Giant Peach (1996),And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001)
6,Friday the 13th Part IV: The Final Chapter (1984),Fly Away Home (1996),Cinderella Man (2005)
7,Slumber Party Massacre III (1990),Space Jam (1996),Girl with a Pearl Earring (2003)
8,Friday the 13th Part 2 (1981),Tin Cup (1996),Eternal Sunshine of the Spotless Mind (2004)
9,"Nightmare on Elm Street 5: The Dream Child, A ...",Star Trek: First Contact (1996),Walk the Line (2005)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

### Static and Dynamic Filters

Lets interact with the static filters we created in the previous notebook, and utilize dynamic filters in realtime.

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata (that arent range based) - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.

Now lets apply item filters to see recommendations for one of these users within each decade of our static filters.


In [16]:
def get_new_recommendations_df_by_static_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [1]:
def get_new_recommendations_df_by_dynamicfilter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = genre_filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given decade. Within a VOD application you could create Shelves (also known as rails or carousels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [18]:
recommendations_df_decade_shelves = pd.DataFrame()
for filter_arn in meta_filter_decade_arns:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_static_filter(recommendations_df_decade_shelves, user, filter_arn)

recommendations_df_decade_shelves

,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,Rear Window (1954),Romeo and Juliet (1968),"Clockwork Orange, A (1971)",NausicaÃ¤ of the Valley of the Wind (Kaze no t...,Schindler's List (1993),Catch Me If You Can (2002),Django Unchained (2012)
1,Vertigo (1958),Lawrence of Arabia (1962),"Godfather: Part II, The (1974)",Dead Poets Society (1989),"Piano, The (1993)",Ocean's Eleven (2001),Inception (2010)
2,"Bridge on the River Kwai, The (1957)","Graduate, The (1967)","Godfather, The (1972)","Professional, The (Le professionnel) (1981)",Titanic (1997),"Beautiful Mind, A (2001)","Dark Knight Rises, The (2012)"
3,12 Angry Men (1957),"Man Who Shot Liberty Valance, The (1962)",Taxi Driver (1976),Scarface (1983),LÃ©on: The Professional (a.k.a. The Profession...,Bowling for Columbine (2002),Interstellar (2014)
4,Oklahoma! (1955),Dr. Strangelove or: How I Learned to Stop Worr...,One Flew Over the Cuckoo's Nest (1975),WarGames (1983),"Usual Suspects, The (1995)",Pride & Prejudice (2005),"King's Speech, The (2010)"
5,North by Northwest (1959),Breakfast at Tiffany's (1961),Apocalypse Now (1979),Raiders of the Lost Ark (Indiana Jones and the...,Sleepless in Seattle (1993),And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),Moonrise Kingdom (2012)
6,From Here to Eternity (1953),Breathless (Ã bout de souffle) (1960),All the President's Men (1976),Star Wars: Episode V - The Empire Strikes Back...,Forrest Gump (1994),Cinderella Man (2005),Harry Potter and the Deathly Hallows: Part 1 (...
7,The Diary of Anne Frank (1959),Doctor Zhivago (1965),Star Wars: Episode IV - A New Hope (1977),Die Hard (1988),Reservoir Dogs (1992),Girl with a Pearl Earring (2003),The Theory of Everything (2014)
8,Marty (1955),"Great Escape, The (1963)",Rocky (1976),Full Metal Jacket (1987),Good Will Hunting (1997),Eternal Sunshine of the Spotless Mind (2004),Her (2013)
9,Quo Vadis (1951),Once Upon a Time in the West (C'era una volta ...,Barry Lyndon (1975),Back to the Future (1985),Seven (a.k.a. Se7en) (1995),Walk the Line (2005),Whiplash (2014)


In [19]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
2,Adventure|Children|Fantasy,1995,0
3,Comedy|Romance,1995,0
4,Comedy|Drama|Romance,1995,0
5,Comedy,1995,0
6,Action|Crime|Thriller,1995,0
7,Comedy|Romance,1995,0
8,Adventure|Children,1995,0
9,Action,1995,0


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [20]:
unique_genre_field_values = items_meta_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [21]:
genres_to_filter

['Film-Noir',
 'Action',
 'Animation',
 'IMAX',
 'Documentary',
 'Mystery',
 'Comedy',
 'Fantasy',
 'Horror',
 'Children',
 'Drama',
 'Crime',
 'Romance',
 'Adventure',
 'Sci-Fi',
 'Musical',
 'Thriller',
 'War',
 'Western']

In [22]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_dynamicfilter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Film-Noir,Action,Animation,IMAX,Documentary,Mystery,Comedy,Fantasy,Horror,Children,Drama,Crime,Romance,Adventure,Sci-Fi,Musical,Thriller,War,Western
0,Sin City (2005),Minority Report (2002),Howl's Moving Castle (Hauru no ugoku shiro) (2...,"Matrix Revolutions, The (2003)",Bowling for Columbine (2002),Minority Report (2002),Lost in Translation (2003),Big Fish (2003),"Others, The (2001)",Shrek (2001),Catch Me If You Can (2002),Catch Me If You Can (2002),"Beautiful Mind, A (2001)","Last Samurai, The (2003)",Eternal Sunshine of the Spotless Mind (2004),Walk the Line (2005),Ocean's Eleven (2001),"Last Samurai, The (2003)",There Will Be Blood (2007)
1,Mulholland Drive (2001),Munich (2005),NausicaÃ¤ of the Valley of the Wind (Kaze no t...,Batman Begins (2005),Fahrenheit 9/11 (2004),21 Grams (2003),Love Actually (2003),Howl's Moving Castle (Hauru no ugoku shiro) (2...,"Ring, The (2002)","Incredibles, The (2004)","Beautiful Mind, A (2001)",Ocean's Eleven (2001),Pride & Prejudice (2005),Howl's Moving Castle (Hauru no ugoku shiro) (2...,Minority Report (2002),Moulin Rouge (2001),Traffic (2000),Schindler's List (1993),Legends of the Fall (1994)
2,L.A. Confidential (1997),"Last Samurai, The (2003)",Shrek (2001),Spider-Man 2 (2004),"Story of the Weeping Camel, The (Geschichte vo...",Mystic River (2003),Bend It Like Beckham (2002),"Lord of the Rings: The Fellowship of the Ring,...",Identity (2003),Harry Potter and the Sorcerer's Stone (a.k.a. ...,Pride & Prejudice (2005),Traffic (2000),And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),War of the Worlds (2005),"Clockwork Orange, A (1971)",Dancer in the Dark (2000),Minority Report (2002),Lord of War (2005),Django Unchained (2012)
3,Chinatown (1974),War of the Worlds (2005),"Incredibles, The (2004)",V for Vendetta (2006),Touching the Void (2003),Donnie Darko (2001),About a Boy (2002),"Lord of the Rings: The Two Towers, The (2002)",28 Days Later (2002),"Chronicles of Narnia: The Lion, the Witch and ...",And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),Minority Report (2002),Cinderella Man (2005),Hero (Ying xiong) (2002),Donnie Darko (2001),Shrek 2 (2004),21 Grams (2003),Atonement (2007),3:10 to Yuma (2007)
4,Brick (2005),Hero (Ying xiong) (2002),WALLÂ·E (2008),I Am Legend (2007),Fog of War: Eleven Lessons from the Life of Ro...,"Usual Suspects, The (1995)",Sideways (2004),Bruce Almighty (2003),I Am Legend (2007),WALLÂ·E (2008),Cinderella Man (2005),Crash (2004),Girl with a Pearl Earring (2003),Serenity (2005),War of the Worlds (2005),Rent (2005),"Clockwork Orange, A (1971)",Hotel Rwanda (2004),"Missing, The (2003)"
5,Dark City (1998),Lord of War (2005),"Monsters, Inc. (2001)",Spider-Man 3 (2007),Super Size Me (2004),Sin City (2005),"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...",NausicaÃ¤ of the Valley of the Wind (Kaze no t...,Saw (2004),"Monsters, Inc. (2001)",Girl with a Pearl Earring (2003),Slumdog Millionaire (2008),Eternal Sunshine of the Spotless Mind (2004),"Motorcycle Diaries, The (Diarios de motociclet...",Serenity (2005),Corpse Bride (2005),Munich (2005),"Pianist, The (2002)",The Alamo (2004)
6,One False Move (1992),"Italian Job, The (2003)",Shrek 2 (2004),300 (2007),Enron: The Smartest Guys in the Room (2005),Old Boy (2003),Something's Gotta Give (2003),Shrek (2001),Battle Royale (Batoru rowaiaru) (2000),Shrek 2 (2004),Eternal Sunshine of the Spotless Mind (2004),21 Grams (2003),Walk the Line (2005),City of God (Cidade de Deus) (2002),Spider-Man (2002),"Phantom of the Opera, The (2004)",Donnie Darko (2001),Enemy at the Gates (2001),"Claim, The (2000)"
7,Lost Highway (1997),Serenity (2005),Corpse Bride (2005),Inception (2010),"Corporation, The (2003)",Mulholland Drive (2001),Garden State (2004),"Lord of the Rings: The Return of the King, The...",Aliens (1986),Ratatouille (2007),Walk the Line (2005),"Clockwork Orange, A (1971)",Brokeback Mountain (2005),"Lord of the Rings: The Fellowship of the Ring,...",Unbreakable (2000),"Lion King, The (1994)",War of the Worlds (2005),"Downfall 

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [23]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:832194813872:event-tracker/dff308a5
b6429d36-b0ec-4921-b5ce-4720add14f2d


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [24]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [25]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,263
0,Catch Me If You Can (2002)
1,Ocean's Eleven (2001)
2,"Beautiful Mind, A (2001)"
3,Bowling for Columbine (2002)
4,Pride & Prejudice (2005)
5,And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001)
6,Cinderella Man (2005)
7,Girl with a Pearl Earring (2003)
8,Eternal Sunshine of the Spotless Mind (2004)
9,Walk the Line (2005)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [26]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [27]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Unicorn City (2012)
sending event click for Trinity and Sartana Are Coming (1972)
sending event click for Hamlet (1964)


Now we can look at how the click events changed the recommendations.

In [28]:
user_recommendations_df

,263,Unicorn City (2012),Trinity and Sartana Are Coming (1972),Hamlet (1964)
0,Catch Me If You Can (2002),Catch Me If You Can (2002),No Strings Attached (2011),Blazing Saddles (1974)
1,Ocean's Eleven (2001),Ocean's Eleven (2001),Easy A (2010),Dog Day Afternoon (1975)
2,"Beautiful Mind, A (2001)","Beautiful Mind, A (2001)",Moonrise Kingdom (2012),Little Big Man (1970)
3,Bowling for Columbine (2002),Bowling for Columbine (2002),"Paris, I Love You (Paris, je t'aime) (2006)",Sixteen Candles (1984)
4,Pride & Prejudice (2005),Pride & Prejudice (2005),"Crazy, Stupid, Love. (2011)","Sting, The (1973)"
5,And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),And Your Mother Too (Y tu mamÃ¡ tambiÃ©n) (2001),Friends with Benefits (2011),"Exorcist, The (1973)"
6,Cinderella Man (2005),Cinderella Man (2005),Life as We Know It (2010),Annie Hall (1977)
7,Girl with a Pearl Earring (2003),Girl with a Pearl Earring (2003),"Adjustment Bureau, The (2011)",Blade Runner (1982)
8,Eternal Sunshine of the Spotless Mind (2004),Eternal Sunshine of the Spotless Mind (2004),"Perks of Being a Wallflower, The (2012)",High Plains Drifter (1973)
9,Walk the Line (2005),Walk the Line (2005),Her (2013),"Clockwork Orange, A (1971)"


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has as a header of the movie that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [30]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Amadeus (1984),2001: A Space Odyssey (1968)
1,Gosford Park (2001),12 Angry Men (1957)
2,"Iron Giant, The (1999)",Cool Hand Luke (1967)
3,Almost Famous (2000),Lawrence of Arabia (1962)
4,Bend It Like Beckham (2002),Seven Samurai (Shichinin no samurai) (1954)
5,American Beauty (1999),"Graduate, The (1967)"
6,Traffic (2000),"Great Escape, The (1963)"
7,Cast Away (2000),All About Eve (1950)
8,Billy Elliot (2000),Raging Bull (1980)
9,Life Is Beautiful (La Vita Ã¨ bella) (1997),Citizen Kane (1941)


Now lets send a watch event in for the top 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [31]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for 2001: A Space Odyssey (1968)
sending event watch for 12 Angry Men (1957)
sending event watch for Cool Hand Luke (1967)
sending event watch for Lawrence of Arabia (1962)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [32]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,"Beautiful Mind, A (2001)","Bridge on the River Kwai, The (1957)"
1,Bowling for Columbine (2002),Paths of Glory (1957)
2,Girl with a Pearl Earring (2003),"Great Escape, The (1963)"
3,Eternal Sunshine of the Spotless Mind (2004),The Diary of Anne Frank (1959)
4,Traffic (2000),"African Queen, The (1951)"
5,Before Sunset (2004),Lifeboat (1944)
6,Lost in Translation (2003),"Battle of Algiers, The (La battaglia di Algeri..."
7,Minority Report (2002),Battleship Potemkin (1925)
8,Big Fish (2003),Stalag 17 (1953)
9,"Aviator, The (2004)",Dr. Strangelove or: How I Learned to Stop Worr...


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [33]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [34]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,"Fast Five (Fast and the Furious 5, The) (2011)"
1,Marooned in Iraq (Gomgashtei dar Aragh) (2002)
2,Themroc (1973)
3,Book of Love (1990)
4,John Mulaney: New In Town (2012)
5,Popeye (1980)
6,Honeymoon (2014)
7,I Am Legend (2007)
8,28 Days (2000)
9,"New Guy, The (2002)"


Then make the personalized ranking API call.

In [35]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [36]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,"Fast Five (Fast and the Furious 5, The) (2011)",Popeye (1980)
1,Marooned in Iraq (Gomgashtei dar Aragh) (2002),Themroc (1973)
2,Themroc (1973),"Castle, The (1997)"
3,Book of Love (1990),Gotti (1996)
4,John Mulaney: New In Town (2012),Fugitives (1986)
5,Popeye (1980),Cabaret (1972)
6,Honeymoon (2014),"New Guy, The (2002)"
7,I Am Legend (2007),I Am Legend (2007)
8,28 Days (2000),Black Beauty (1994)
9,"New Guy, The (2002)",28 Days (2000)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/recommendations-batch.html). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": <S3 input path>}},
    jobOutput = 
       {"s3DataDestination": {"path": <S3 output path>}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [89]:
# We will use the same users from before
print (users)
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [90]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "320"}
{"userId": "403"}
{"userId": "59"}


Upload the file to S3 and save the path as a variable for later.

In [91]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://832194813872-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [92]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://832194813872-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [93]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [94]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  05:13:33 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [95]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 320,User: 403,User: 59
0,X2: X-Men United (2003),"Hangover, The (2009)",Network (1976)
1,"Matrix Revolutions, The (2003)",Dr. Strangelove or: How I Learned to Stop Worr...,American Graffiti (1973)
2,Avatar (2009),Clueless (1995),Chinatown (1974)
3,Spider-Man 2 (2004),LÃ©on: The Professional (a.k.a. The Profession...,This Is Spinal Tap (1984)
4,Iron Man (2008),Forgetting Sarah Marshall (2008),"Sting, The (1973)"
5,Mr. & Mrs. Smith (2005),Juno (2007),Gandhi (1982)
6,Spider-Man (2002),Reservoir Dogs (1992),Raging Bull (1980)
7,"Hitchhiker's Guide to the Galaxy, The (2005)",Borat: Cultural Learnings of America for Make ...,Amadeus (1984)
8,Fantastic Four (2005),Sleepless in Seattle (1993),Trainspotting (1996)
9,Serenity (2005),Mr. & Mrs. Smith (2005),"Big Chill, The (1983)"


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, make a pull request and provide any additional helpful components that may be missing from this collection.

You can choose to head to `04_Operations_Layer.ipynb` to go deeper into ML Ops and what a production solution can look like with an automation pipeline.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `05_Clean_Up.ipynb`.

In [37]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
